In [45]:
import numpy as np
import pandas as pd
import scipy.stats as stats

def generate_patient_vitals(id, n, t=3):
    np.random.seed(id)
    
    # Simulate random values for heart rate (bpm)
    heart_rate = np.round(stats.truncnorm.rvs(-t, t, 72, 12, n), 2)
    
    # Simulate random values for blood pressure (mmHg)
    systolic_bp = np.round(stats.truncnorm.rvs(-t, t, 115, 12, n), 2)
    diastolic_bp = np.round(stats.truncnorm.rvs(-t, t, 75, 10, n), 2)

    # Simulate random values for temperature (°C)
    temperature = np.round(stats.truncnorm.rvs(-t, t, 37, 0.5, n), 2)

    # Simulate random values for respiration rate (breaths per minute)
    respiration_rate = np.round(stats.truncnorm.rvs(-t, t, 17, 3, n), 2)

    # Simulate random values for SpO2 (%)
    spo2 = np.round(stats.truncnorm.rvs(-t, t, 98, 1.5, n), 2)
    spo2[spo2 > 100] = 100

    return pd.DataFrame({
        'Patient ID': id,
        'Heart Rate': heart_rate,
        'Systolic BP': systolic_bp,
        'Diastolic BP': diastolic_bp,
        'Temperature': temperature,
        'Respiration Rate': respiration_rate,
        'SpO2': spo2
    })

In [46]:
import json

# Processing address dataset
geojson_file_path = '../ma_addresses/city_of_boston-addresses-city.geojson'

# Extract relevant information
addresses = []
with open(geojson_file_path, 'r') as file:
    for line in file:
        try:
            data = json.loads(line)
            properties = data.get("properties", {})
            number = properties.get("number", "")
            street = properties.get("street", "")
            postcode = properties.get("postcode", "")

            # Create consolidated address
            consolidated_address = f"{number} {street}, {postcode}"

            # Append to addresses list
            addresses.append(consolidated_address)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Create a DataFrame
address_df = pd.DataFrame({"Address": addresses})

# Checking Addresses
address_df.head()

,Address
0,"6-10 A Street, 02136"
1,"7 A Street, 02136"
2,"10 A Street, 02127"
3,"172-174 A Street, 02210"
4,"176-178 A Street, 02210"


In [47]:
Patient_count = 10

In [48]:
patient_df = pd.DataFrame([])
for i in range(1, Patient_count+1):
    patient_df = pd.concat([patient_df, generate_patient_vitals(i, 100)])
address_df_sample = address_df.sample(n = Patient_count).reset_index()    

In [49]:
address_df_sample

,index,Address
0,194005,"68 W Concord Street, 02118"
1,92615,"142 Norwell Street, 02121"
2,34143,"1666 Commonwealth Avenue, 02135"
3,187283,"52 Columbia Road, 02121"
4,317145,"456 Centre Street, 02130"
5,219258,"38 Greenough Avenue, 02130"
6,143928,"1086-1088A Dorchester Avenue, 02125"
7,47382,"30 Elm Street, 02130"
8,243300,"73 Maple Street, 02136"
9,147641,"69-69A Readville Street, 02136"


In [50]:
from faker import Faker
import random

fake = Faker()

def generate_random_phone_number():
    # Generate a random US phone number for Boston area code (617)
    area_code = "617"
    middle_digits = f"{random.randint(200, 999):03d}"
    last_digits = f"{random.randint(1000, 9999):04d}"
    return f"({area_code}) {middle_digits}-{last_digits}"

def generate_random_ward_number():
    # Generate a random ward number (assuming a range)
    return random.randint(1, 20)

# Generate random patients with names, phone numbers, and ward numbers
names = []
phnumber = []
ward_list = []
for _ in range(Patient_count):
    names.append(fake.name())
    phnumber.append(generate_random_phone_number())
    ward_list.append(generate_random_ward_number())
    
detail_df = pd.DataFrame({"Patient Name": names, "Phone Number": phnumber, "Ward Number": ward_list})
detail_df

,Patient Name,Phone Number,Ward Number
0,David Hatfield,(617) 347-2993,11
1,Linda Morales,(617) 884-5088,11
2,Melissa Welch,(617) 383-9205,4
3,Tammy King,(617) 882-7110,3
4,Johnny Anderson,(617) 532-9280,14
5,Jean Jones,(617) 917-9888,7
6,Wyatt Macdonald,(617) 856-2370,12
7,Lauren Harmon,(617) 839-2087,19
8,Scott Rodriguez,(617) 780-6962,17
9,Shawn Montgomery,(617) 616-3500,7


In [51]:
# Joining detail DF and addresses DF
patient_info_df = detail_df.join(address_df_sample)
patient_info_df

,Patient Name,Phone Number,Ward Number,index,Address
0,David Hatfield,(617) 347-2993,11,194005,"68 W Concord Street, 02118"
1,Linda Morales,(617) 884-5088,11,92615,"142 Norwell Street, 02121"
2,Melissa Welch,(617) 383-9205,4,34143,"1666 Commonwealth Avenue, 02135"
3,Tammy King,(617) 882-7110,3,187283,"52 Columbia Road, 02121"
4,Johnny Anderson,(617) 532-9280,14,317145,"456 Centre Street, 02130"
5,Jean Jones,(617) 917-9888,7,219258,"38 Greenough Avenue, 02130"
6,Wyatt Macdonald,(617) 856-2370,12,143928,"1086-1088A Dorchester Avenue, 02125"
7,Lauren Harmon,(617) 839-2087,19,47382,"30 Elm Street, 02130"
8,Scott Rodriguez,(617) 780-6962,17,243300,"73 Maple Street, 02136"
9,Shawn Montgomery,(617) 616-3500,7,147641,"69-69A Readville Street, 02136"


In [52]:
patient_df.columns

Index(['Patient ID', 'Heart Rate', 'Systolic BP', 'Diastolic BP',
       'Temperature', 'Respiration Rate', 'SpO2'],
      dtype='object')

In [53]:
patient_df.describe()

,Patient ID,Heart Rate,Systolic BP,Diastolic BP,Temperature,Respiration Rate,SpO2
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5.500000,71.757730,114.64872,75.003520,37.010400,17.047100,97.868930
std,2.873719,11.320872,12.03627,9.678028,0.496171,2.943674,1.441492
min,1.000000,36.300000,80.63000,45.770000,35.650000,8.490000,93.520000
25%,3.000000,64.595000,106.19500,68.525000,36.690000,15.140000,96.850000
50%,5.500000,71.860000,115.09500,75.115000,37.015000,17.075000,97.930000
75%,8.000000,78.985000,123.11250,81.900000,37.350000,19.045000,99.012500
max,10.000000,101.410000,146.80000,101.250000,38.430000,25.300000,100.000000


In [54]:
patient_df.to_csv("patient_vitals_CSV/patient_vitals.csv")
patient_info_df.to_csv("patient_vitals_CSV/patient_info.csv")